In [ ]:
import pickle
import pandas as pd
import io
import numpy as np
import nltk
from nltk.corpus import stopwords
import math
import re
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
articles = pd.read_csv('data/articles.csv')

In [ ]:
df = pd.read_csv('data/train.csv')

In [ ]:
top10_pred_path = pickle.load(open('./tmp/Challenge_Dataset/top10_pred_paths.dat', 'rb'))

In [ ]:
def index_set(s):
    i = 0
    s_map = {}
    for key in s:
        s_map[key] = str(i)
        i+=1
    return s_map

In [ ]:
user_id = df['qid'].unique()
article_id=articles['article_id'].unique()
topic_list = []
topics = articles['topic'].dropna()
for topic in topics:
    t = topic.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_list:
            topic_list.append(j)
topic_tag_list = []
topic_tags = articles['vstopic'].dropna()
for tag in topic_tags:
    t = tag.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in topic_tag_list:
            topic_tag_list.append(j)
product_list = []
products = articles['product'].dropna()
for product in products:
    t = product.strip('[').strip(']').strip().split(',')
    for i in range(len(t)):
        j = t[i].strip().replace("'",'')
        if j not in product_list:
            product_list.append(j)

In [ ]:
user_map = index_set(user_id)
article_map = index_set(article_id)
topic_map = index_set(topic_list)
tag_map = index_set(topic_tag_list)
product_map = index_set(product_list)

In [ ]:
user_map_trans = {v: k for k,v in user_map.items()}
article_map_trans = {v: k for k,v in article_map.items()}
product_map_trans = {v: k for k,v in product_map.items()}
topic_map_trans = {v: k for k,v in topic_map.items()}
tag_map_trans = {v: k for k,v in tag_map.items()}

In [ ]:
pred_real_article = pickle.load(open('./tmp/Challenge_Dataset/pred_real_article.dat', 'rb'))

In [ ]:
dataset = pd.DataFrame()

In [ ]:
for i in range(len(pred_real_article)):
    pred_l_idx = re.search('pred_list:',pred_real_article[i]).span()
    real_l_idx = re.search('rel_set:',pred_real_article[i]).span()
    uid = pred_real_article[i][:pred_l_idx[0]].replace('uid:','').replace(' ','')
    pred_l = list(pred_real_article[i][pred_l_idx[0]:real_l_idx[0]].replace('pred_list:','').replace('[','').replace(']','').replace(' ','').split(','))
    real_l = list(pred_real_article[i][real_l_idx[0]:].replace('rel_set:','').replace('[','').replace(']','').replace(' ','').split(','))
    dataset = dataset.append([{'uid':uid,'pred':pred_l,'real':real_l}])

<ipython-input-14-2de862811f7b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([{'uid':uid,'pred':pred_l,'real':real_l}])
<ipython-input-14-2de862811f7b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([{'uid':uid,'pred':pred_l,'real':real_l}])
<ipython-input-14-2de862811f7b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([{'uid':uid,'pred':pred_l,'real':real_l}])
<ipython-input-14-2de862811f7b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append([{'uid':uid,'pred':pred_l,'real':real_l}])
<ipython-input-14-2de862811f7b>:7: FutureWarning

In [ ]:
pred_real = pd.DataFrame()
for index, row in dataset.iterrows():
    pred, real = [], []
    for a in row['pred']:
        pred.append(article_map_trans[a])
    for a in row['real']:
        real.append(article_map_trans[a])
    pred_real = pred_real.append([{'uid':user_map_trans[str(row['uid'])],'pred':pred,'real':real}])

<ipython-input-15-a05dce05a83a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_real = pred_real.append([{'uid':user_map_trans[str(row['uid'])],'pred':pred,'real':real}])
<ipython-input-15-a05dce05a83a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_real = pred_real.append([{'uid':user_map_trans[str(row['uid'])],'pred':pred,'real':real}])
<ipython-input-15-a05dce05a83a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_real = pred_real.append([{'uid':user_map_trans[str(row['uid'])],'pred':pred,'real':real}])
<ipython-input-15-a05dce05a83a>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_real = pred_real.append([{'

In [ ]:
best_pred_path_trans={}
for k,v in top10_pred_path.items():
    k_d = k
    for i in v[0]:
        j = i[len(i)-1]
        t = 0
        for k in j:
            if k[1]=='user':
                k = list(k)
                k[2]=user_map_trans[str(k[2])]
                k = tuple(k)
            if k[1]=='article':
                k = list(k)
                k[2]=article_map_trans[str(k[2])]
                k = tuple(k)
            if k[1]=='related_article':
                k = list(k)
                k[2]=article_map_trans[str(k[2])]
                k = tuple(k)
            if k[1]=='product':
                k = list(k)
                k[2]=product_map_trans[str(k[2])]
            if k[1]=='topic':
                k = list(k)
                k[2]=topic_map_trans[str(k[2])]
            if k[1]=='tag':
                k = list(k)
                k[2]=tag_map_trans[str(k[2])]
            j[t]=k
            t+=1
        i = list(i)
        i[2]=j
        i = tuple(i)
    best_pred_path_trans[user_map_trans[str(k_d)]]=v[0]

In [ ]:
def word_dict_generate(text):
    words = []
    for line in text:
        line = line.strip().lower()
        line = re.sub(r","," ",line)
        line = re.sub(r";"," ",line)
        line = re.sub(r"\'"," ",line)
        line = re.sub(r":"," ",line)
        line = re.sub(r"!"," ",line)
        line = re.sub(r"\."," ",line)
        line = re.sub(r"\/"," ",line)
        line = re.sub(r"\?"," ", line)
        line = re.sub(r"[\\d]"," ",line)
        line = re.sub(r"\'\'"," ",line)
        line = re.sub(r"\("," ",line)
        line = re.sub(r"\)"," ",line)
        line = re.sub(r"\<"," ",line)
        line = re.sub(r"\>"," ",line)
        line = nltk.word_tokenize(line)
        for w in line:
            if w not in stopwords.words("english") and len(w)>3:
                words.append(w)
    return words

In [ ]:
def text_clean(text):
    line = text.strip()
    line = re.sub(r";","",line)
    line = re.sub(r"\'","",line)
    line = re.sub(r":","",line)
    line = re.sub(r"!","",line)
    line = re.sub(r"\.","",line)
    line = re.sub(r"\/","",line)
    line = re.sub(r"\?","", line)
    line = re.sub(r"\'\'","",line)
    line = re.sub(r"\(","",line)
    line = re.sub(r"\)","",line)
    line = re.sub(r"\<","",line)
    line = re.sub(r"\>","",line)
    return line

In [ ]:
def common_words_generate(tcm_ids):
    common_words = []
    if len(tcm_ids) >= 2:
        a_1 = articles[articles['article_id'] == tcm_ids[0]]['text']
        w_1 = word_dict_generate(a_1)
        a_2 = articles[articles['article_id'] == tcm_ids[1]]['text']
        w_2 = word_dict_generate(a_2)
        common_words = list(set(w_1).intersection(set(w_2)))
        if len(tcm_ids) > 2:
            for t in range(2,len(tcm_ids)):
                a_t = articles[articles['article_id'] == tcm_ids[t]]['text']
                w_t = word_dict_generate(a_t)
                common_words = list(set(common_words).intersection(set(w_t)))
    return common_words

In [ ]:
def common_words_p_t(tcm_ids,s):
    r = []
    a_1 = articles[articles['article_id'] == tcm_ids[0]][s].to_string(index = False).strip('[').strip(']').strip().split(',')
    a_1 = [text_clean(i) for i in a_1 if i!='NaN']
    a_2 = articles[articles['article_id'] == tcm_ids[1]][s].to_string(index = False).strip('[').strip(']').strip().split(',')
    a_2 = [text_clean(i) for i in a_2 if i!='NaN']
    r = list(set(a_1).intersection(set(a_2)))
    if len(tcm_ids) > 2:
        for t in range(2,len(tcm_ids)):
            a_t = articles[articles['article_id'] == tcm_ids[t]][s].to_string(index = False).strip('[').strip(']').strip().split(',')
            a_t = [i.replace('\'','') for i in a_t]
            a_t = [text_clean(i) for i in a_t if i!='NaN']
            r = list(set(r).intersection(set(a_t)))
    return r

In [ ]:
common_words = pd.DataFrame()

In [ ]:
for uid in pred_real['uid']:
    for i in range(len(best_pred_path_trans[uid])):
        tcm_ids = []
        for t in best_pred_path_trans[uid][i][2]:
            if t[1] == 'article' or t[1] == 'related_article':
                tcm_ids.append(t[2])
        c_w = common_words_generate(tcm_ids)
        c_p = common_words_p_t(tcm_ids,'product')
        c_t = common_words_p_t(tcm_ids,'topic')

        common_words = common_words.append([{'uid':uid,'article_ids':tcm_ids,'commutative score':best_pred_path_trans[uid][i][0],'score':best_pred_path_trans[uid][i][1],'product':c_p,'topic':c_t,'common_words':c_w}])

<ipython-input-23-15e670376216>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  common_words = common_words.append([{'uid':uid,'article_ids':tcm_ids,'commutative score':best_pred_path_trans[uid][i][0],'score':best_pred_path_trans[uid][i][1],'product':c_p,'topic':c_t,'common_words':c_w}])
<ipython-input-23-15e670376216>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  common_words = common_words.append([{'uid':uid,'article_ids':tcm_ids,'commutative score':best_pred_path_trans[uid][i][0],'score':best_pred_path_trans[uid][i][1],'product':c_p,'topic':c_t,'common_words':c_w}])
<ipython-input-23-15e670376216>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  common_words = common_words.append([{'uid':uid,'article_ids':tcm_ids,

In [ ]:
common_words.to_csv('./path_with_common_words.csv',index=False)